## BAF Veri Seti İnceleme

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.ensemble import RandomForestClassifier

C:\Users\Beyza\anaconda3\anacondaa3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Beyza\anaconda3\anacondaa3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv("sample10.csv")

In [3]:
df

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month,fraud_bool
0,0.1,0.423184,-1,174,70,0.022402,3.354768,AA,1140,3659.730906,...,200.0,0,INTERNET,14.859346,windows,1,1,0,0,0
1,0.7,0.246312,-1,60,50,0.009289,-0.305261,AB,1118,4950.849565,...,500.0,0,INTERNET,2.320669,linux,0,1,0,5,0
2,0.7,0.921065,29,148,40,0.009191,-1.054443,AD,399,574.335478,...,1500.0,0,INTERNET,7.387905,windows,1,1,0,6,0
3,0.7,0.547707,56,8,20,0.004642,-1.124991,AB,4062,3484.847833,...,200.0,0,INTERNET,27.903481,macintosh,1,1,0,6,0
4,0.9,0.860882,-1,46,20,0.005121,-1.067835,AD,1052,14073.211653,...,200.0,0,INTERNET,5.674654,windows,1,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.7,0.132291,-1,24,40,0.033752,43.911131,AA,1086,1587.867880,...,200.0,0,INTERNET,9.466239,windows,0,1,0,7,0
9996,0.9,0.637731,-1,37,50,0.019209,-1.429492,AB,2494,6708.114438,...,200.0,0,INTERNET,2.081959,other,1,1,0,3,0
9997,0.8,0.529391,54,1,30,0.003783,13.687813,AA,1448,902.032557,...,500.0,0,INTERNET,4.601284,linux,0,1,0,7,0
9998,0.8,0.224031,-1,39,40,0.007705,-0.427604,AC,385,4331.426764,...,200.0,0,INTERNET,2.502733,other,1,1,0,7,0


In [4]:
def col_names(dataframe, cat_th = 200):
    
    
    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ["category", "bool", "object"]]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]

    cat_cols = cat_cols + num_but_cat
    
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]
    
    return cat_cols, num_cols

cat_cols, num_cols = col_names(df)

In [5]:
cat_cols

['payment_type',
 'employment_status',
 'housing_status',
 'source',
 'device_os',
 'income',
 'customer_age',
 'date_of_birth_distinct_emails_4w',
 'email_is_free',
 'phone_home_valid',
 'phone_mobile_valid',
 'bank_months_count',
 'has_other_cards',
 'proposed_credit_limit',
 'foreign_request',
 'keep_alive_session',
 'device_distinct_emails_8w',
 'device_fraud_count',
 'month',
 'fraud_bool']

In [6]:
lelist = []

def display_value_counts(dataframe):
    for col in dataframe.columns:
        unique_values_count = dataframe[col].nunique()
        if unique_values_count>2 and unique_values_count <= 40:
            lelist.append(col)
            
display_value_counts(df)

In [7]:
lelist

['income',
 'customer_age',
 'payment_type',
 'date_of_birth_distinct_emails_4w',
 'employment_status',
 'housing_status',
 'bank_months_count',
 'proposed_credit_limit',
 'device_os',
 'device_distinct_emails_8w',
 'month']

In [8]:
categorical_columns = ['income', 'customer_age', 'payment_type', 'employment_status', 'housing_status', 'source', 'device_os', 'device_distinct_emails_8w', 'device_fraud_count', 'month']

In [9]:
one_hot_encoded_df = pd.get_dummies(df, columns=['income', 'customer_age', 'payment_type', 'employment_status', 'housing_status', 'source', 'device_os', 'device_distinct_emails_8w', 'device_fraud_count', 'month'])
target_column = 'fraud_bool'

one_hot_encoded_data = one_hot_encoded_df.drop(columns=[target_column])

boolean_columns = one_hot_encoded_df.select_dtypes(include=['bool']).columns
for column in boolean_columns:
    one_hot_encoded_df[column] = one_hot_encoded_df[column].astype(int)
    
target_data = one_hot_encoded_df[target_column]

In [10]:
one_hot_encoded_df.columns

Index(['name_email_similarity', 'prev_address_months_count',
       'current_address_months_count', 'days_since_request',
       'intended_balcon_amount', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
       'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'credit_risk_score',
       'email_is_free', 'phone_home_valid', 'phone_mobile_valid',
       'bank_months_count', 'has_other_cards', 'proposed_credit_limit',
       'foreign_request', 'session_length_in_minutes', 'keep_alive_session',
       'fraud_bool', 'income_0.1', 'income_0.2', 'income_0.3', 'income_0.4',
       'income_0.5', 'income_0.6000000000000001', 'income_0.7000000000000001',
       'income_0.8', 'income_0.9', 'customer_age_10', 'customer_age_20',
       'customer_age_30', 'customer_age_40', 'customer_age_50',
       'customer_age_60', 'customer_age_70', 'customer_age_80',
       'customer_age_90', 'payment_type_AA', 'payment_type_AB',
       'payment_type_AC', 'payment_type_AD', 'paymen

In [11]:
selected_columns=['name_email_similarity', 'prev_address_months_count',
       'current_address_months_count', 'days_since_request',
       'intended_balcon_amount', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
       'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'credit_risk_score',
       'email_is_free', 'phone_home_valid', 'phone_mobile_valid',
       'bank_months_count', 'has_other_cards', 'proposed_credit_limit',
       'foreign_request', 'session_length_in_minutes', 'keep_alive_session',
       'fraud_bool', 'income_0.1', 'income_0.2', 'income_0.3', 'income_0.4',
       'income_0.5', 'income_0.6000000000000001', 'income_0.7000000000000001',
       'income_0.8', 'income_0.9', 'customer_age_10', 'customer_age_20',
       'customer_age_30', 'customer_age_40', 'customer_age_50',
       'customer_age_60', 'customer_age_70', 'customer_age_80',
       'customer_age_90', 'payment_type_AA', 'payment_type_AB',
       'payment_type_AC', 'payment_type_AD', 'payment_type_AE',
       'employment_status_CA', 'employment_status_CB', 'employment_status_CC',
       'employment_status_CD', 'employment_status_CE', 'employment_status_CF',
       'employment_status_CG', 'housing_status_BA', 'housing_status_BB',
       'housing_status_BC', 'housing_status_BD', 'housing_status_BE',
       'housing_status_BF', 'housing_status_BG', 'source_INTERNET',
       'source_TELEAPP', 'device_os_linux', 'device_os_macintosh',
       'device_os_other', 'device_os_windows', 'device_os_x11',
       'device_distinct_emails_8w_-1', 'device_distinct_emails_8w_0',
       'device_distinct_emails_8w_1', 'device_distinct_emails_8w_2',
       'device_fraud_count_0', 'month_0', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7']

new_df = one_hot_encoded_df[selected_columns]

one_hot_encoded_df.drop(['name_email_similarity', 'prev_address_months_count',
       'current_address_months_count', 'days_since_request',
       'intended_balcon_amount', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
       'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'credit_risk_score',
       'email_is_free', 'phone_home_valid', 'phone_mobile_valid',
       'bank_months_count', 'has_other_cards', 'proposed_credit_limit',
       'foreign_request', 'session_length_in_minutes', 'keep_alive_session',
       'fraud_bool', 'income_0.1', 'income_0.2', 'income_0.3', 'income_0.4',
       'income_0.5', 'income_0.6000000000000001', 'income_0.7000000000000001',
       'income_0.8', 'income_0.9', 'customer_age_10', 'customer_age_20',
       'customer_age_30', 'customer_age_40', 'customer_age_50',
       'customer_age_60', 'customer_age_70', 'customer_age_80',
       'customer_age_90', 'payment_type_AA', 'payment_type_AB',
       'payment_type_AC', 'payment_type_AD', 'payment_type_AE',
       'employment_status_CA', 'employment_status_CB', 'employment_status_CC',
       'employment_status_CD', 'employment_status_CE', 'employment_status_CF',
       'employment_status_CG', 'housing_status_BA', 'housing_status_BB',
       'housing_status_BC', 'housing_status_BD', 'housing_status_BE',
       'housing_status_BF', 'housing_status_BG', 'source_INTERNET',
       'source_TELEAPP', 'device_os_linux', 'device_os_macintosh',
       'device_os_other', 'device_os_windows', 'device_os_x11',
       'device_distinct_emails_8w_-1', 'device_distinct_emails_8w_0',
       'device_distinct_emails_8w_1', 'device_distinct_emails_8w_2',
       'device_fraud_count_0', 'month_0', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7'], axis=1, inplace=True)



In [14]:
da = pd.read_csv("sample10.csv")

In [15]:
da.head()

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month,fraud_bool
0,0.1,0.423184,-1,174,70,0.022402,3.354768,AA,1140,3659.730906,...,200.0,0,INTERNET,14.859346,windows,1,1,0,0,0
1,0.7,0.246312,-1,60,50,0.009289,-0.305261,AB,1118,4950.849565,...,500.0,0,INTERNET,2.320669,linux,0,1,0,5,0
2,0.7,0.921065,29,148,40,0.009191,-1.054443,AD,399,574.335478,...,1500.0,0,INTERNET,7.387905,windows,1,1,0,6,0
3,0.7,0.547707,56,8,20,0.004642,-1.124991,AB,4062,3484.847833,...,200.0,0,INTERNET,27.903481,macintosh,1,1,0,6,0
4,0.9,0.860882,-1,46,20,0.005121,-1.067835,AD,1052,14073.211653,...,200.0,0,INTERNET,5.674654,windows,1,1,0,3,0


In [18]:
one_hot_encoded_df = 

Series([], dtype: object)

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

# Veri setini yükleyin
df = pd.read_csv('sample10.csv')

# One-hot encoding işlemi
one_hot_encoded_df = pd.get_dummies(df, columns=['income', 'customer_age', 'payment_type', 'employment_status', 'housing_status', 'source', 'device_os', 'device_distinct_emails_8w', 'device_fraud_count', 'month'])
target_column = 'fraud_bool'

# Kategorik sütunları int formatına dönüştür
boolean_columns = one_hot_encoded_df.select_dtypes(include=['bool']).columns
for column in boolean_columns:
    one_hot_encoded_df[column] = one_hot_encoded_df[column].astype(int)

# Hedef ve özellikleri ayırın
target_data = one_hot_encoded_df[target_column]
features_data = one_hot_encoded_df.drop(columns=[target_column])

# PyTorch veri seti tanımlaması
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data.values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Eğitim ve test setlerine ayırın
X_train, X_test, y_train, y_test = train_test_split(features_data, target_data, test_size=0.2, random_state=42)

# PyTorch veri yükleyicilerini oluşturun
train_dataset = CustomDataset(pd.DataFrame(X_train))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Basit GAN model tanımlamaları
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.fc(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.fc(x)

# GAN'ı oluşturun
latent_dim = 100
input_dim = X_train.shape[1]

generator = Generator(latent_dim, input_dim)
discriminator = Discriminator(input_dim)

criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# GAN eğitimi
num_epochs = 20
for epoch in range(num_epochs):
    for real_data in train_loader:
        batch_size = real_data.size(0)
        real_data = real_data

        # Gerçek ve sahte veriler için etiketler
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        # Discriminator'ı eğitme
        optimizer_d.zero_grad()
        outputs = discriminator(real_data)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        z = torch.randn(batch_size, latent_dim)
        fake_data = generator(z)
        outputs = discriminator(fake_data.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()
        optimizer_d.step()

        # Generator'ı eğitme
        optimizer_g.zero_grad()
        outputs = discriminator(fake_data)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_g.step()

    print(f'Epoch [{epoch+1}/{num_epochs}] | D Loss: {d_loss_real.item() + d_loss_fake.item()} | G Loss: {g_loss.item()}')

# Yapay veri üretme
z = torch.randn(1000, latent_dim)
generated_data = generator(z).detach().numpy()

# Yapay veriyi DataFrame'e dönüştürme
generated_df = pd.DataFrame(generated_data, columns=features_data.columns)

print(generated_df.head())



Epoch [1/20] | D Loss: 0.5418858528137207 | G Loss: 0.8815749287605286
Epoch [2/20] | D Loss: 0.4625571370124817 | G Loss: 1.0124119520187378
Epoch [3/20] | D Loss: 0.28393253684043884 | G Loss: 1.4263298511505127
Epoch [4/20] | D Loss: 0.1794987916946411 | G Loss: 1.8424502611160278
Epoch [5/20] | D Loss: 0.10666599124670029 | G Loss: 2.3356165885925293
Epoch [6/20] | D Loss: 0.0734209418296814 | G Loss: 2.679732322692871
Epoch [7/20] | D Loss: 0.051019806414842606 | G Loss: 3.043825626373291
Epoch [8/20] | D Loss: 0.029141435399651527 | G Loss: 3.5924594402313232
Epoch [9/20] | D Loss: 0.022668126970529556 | G Loss: 3.8441195487976074
Epoch [10/20] | D Loss: 0.026455286890268326 | G Loss: 3.7023191452026367
Epoch [11/20] | D Loss: 0.0186767540872097 | G Loss: 4.033329010009766
Epoch [12/20] | D Loss: 0.01311916671693325 | G Loss: 4.382405757904053
Epoch [13/20] | D Loss: 0.010659908875823021 | G Loss: 4.594203948974609
Epoch [14/20] | D Loss: 0.008495701476931572 | G Loss: 4.8121685

In [3]:
generated_df

,name_email_similarity,prev_address_months_count,current_address_months_count,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,...,device_distinct_emails_8w_2,device_fraud_count_0,month_0,month_1,month_2,month_3,month_4,month_5,month_6,month_7
0,0.825775,0.279056,-0.927691,0.508359,0.852972,-0.180177,0.999976,0.999999,0.999987,0.988954,...,-0.861674,0.316761,-0.874780,-0.670197,-0.479032,0.680742,-0.161259,0.182609,-0.824490,-0.645934
1,0.831336,0.178981,-0.930040,0.478352,0.814562,-0.273523,0.999920,0.999994,0.999973,0.982878,...,-0.816172,0.255279,-0.880072,-0.615725,-0.401183,0.648992,-0.135717,0.094605,-0.812299,-0.635370
2,0.775729,0.265411,-0.924829,0.396761,0.814946,-0.315211,0.999885,0.999990,0.999937,0.977271,...,-0.821022,0.245103,-0.847431,-0.562213,-0.446083,0.680362,-0.023569,0.140835,-0.769631,-0.521531
3,0.705182,0.171469,-0.896053,0.410057,0.755834,-0.196337,0.999502,0.999957,0.999793,0.964609,...,-0.778155,0.184926,-0.818635,-0.597941,-0.317129,0.556708,-0.107244,0.150038,-0.693072,-0.408100
4,0.738986,0.273002,-0.886189,0.501756,0.747789,-0.320675,0.999798,0.999972,0.999867,0.968536,...,-0.793486,0.248058,-0.818897,-0.644491,-0.325813,0.626269,-0.011516,0.122209,-0.724823,-0.525631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.808380,0.245104,-0.919165,0.489541,0.807533,-0.278891,0.999869,0.999988,0.999940,0.975148,...,-0.809053,0.239730,-0.836654,-0.597988,-0.308538,0.653383,-0.074806,0.172754,-0.783383,-0.554076
996,0.756714,0.176667,-0.887982,0.401943,0.755773,-0.194136,0.999699,0.999964,0.999807,0.970908,...,-0.785684,0.175717,-0.823244,-0.484824,-0.341968,0.576152,-0.045706,0.028252,-0.694438,-0.547459
997,0.764970,0.241045,-0.896944,0.336297,0.763764,-0.216694,0.999758,0.999972,0.999870,0.971027,...,-0.770545,0.222777,-0.826813,-0.631227,-0.375274,0.600312,-0.172998,0.192603,-0.724622,-0.469698
998,0.755970,0.257859,-0.902372,0.296679,0.723762,-0.167248,0.999626,0.999942,0.999764,0.966141,...,-0.728309,0.213176,-0.775978,-0.466354,-0.399600,0.541698,-0.124955,0.060648,-0.712596,-0.468622


In [5]:

# Eşik değeri (0.5) üzerinden ikili (binary) hale getirmek istediğiniz sütunları seçin
binary_columns = [
        'income_0.1', 'income_0.2', 'income_0.3', 'income_0.4',
       'income_0.5', 'income_0.6000000000000001', 'income_0.7000000000000001',
       'income_0.8', 'income_0.9', 'customer_age_10', 'customer_age_20',
       'customer_age_30', 'customer_age_40', 'customer_age_50',
       'customer_age_60', 'customer_age_70', 'customer_age_80',
       'customer_age_90', 'payment_type_AA', 'payment_type_AB',
       'payment_type_AC', 'payment_type_AD', 'payment_type_AE',
       'employment_status_CA', 'employment_status_CB', 'employment_status_CC',
       'employment_status_CD', 'employment_status_CE', 'employment_status_CF',
       'employment_status_CG', 'housing_status_BA', 'housing_status_BB',
       'housing_status_BC', 'housing_status_BD', 'housing_status_BE',
       'housing_status_BF', 'housing_status_BG', 'source_INTERNET',
       'source_TELEAPP', 'device_os_linux', 'device_os_macintosh',
       'device_os_other', 'device_os_windows', 'device_os_x11',
       'device_distinct_emails_8w_-1', 'device_distinct_emails_8w_0',
       'device_distinct_emails_8w_1', 'device_distinct_emails_8w_2',
       'device_fraud_count_0', 'month_0', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7']

# İlgili sütunlardaki değerleri 0.5'e göre ikili (binary) hale getirme
generated_df[binary_columns] = generated_df[binary_columns].applymap(lambda x: 1 if x > 0.5 else 0)

generated_df


C:\Users\Beyza\AppData\Local\Temp\ipykernel_12932\2753747745.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  generated_df[binary_columns] = generated_df[binary_columns].applymap(lambda x: 1 if x > 0.5 else 0)


,name_email_similarity,prev_address_months_count,current_address_months_count,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,...,month_5,month_6,month_7,payment_type_AA,payment_type_AB,payment_type_AC,payment_type_AD,payment_type_AE,source_INTERNET,source_TELEAPP
0,0.258326,55.100857,73.546875,-0.730430,6.918195,1304.980835,8024.230957,5844.652344,5546.629883,-238.667709,...,0,1,0,1,0,1,0,0,1,0
1,0.320405,56.863998,34.139175,-3.391500,-10.188081,1888.504517,2954.628906,3363.331055,4091.190918,57.797241,...,0,1,1,0,1,0,0,1,1,0
2,0.516030,38.475037,131.727371,-3.190106,-0.940820,2028.784912,4159.578613,4887.316895,4604.526367,37.729118,...,0,1,1,0,1,0,0,0,1,0
3,0.486139,60.317142,5.642218,-3.792231,-11.002547,1291.702271,3287.737793,3748.450928,4903.172852,-248.362808,...,0,1,1,0,1,0,0,1,1,0
4,0.657974,17.441010,171.707474,-4.487044,-1.563043,2558.560791,2742.557129,3840.071533,4242.323730,389.209991,...,0,1,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.618908,60.605434,8.352049,-3.842474,-11.272525,621.627136,2879.943604,3462.335205,4106.711426,-271.491882,...,0,1,1,0,1,1,0,0,1,0
996,0.524920,27.115891,162.356964,-3.967557,-4.156897,2317.308350,3324.681152,4132.930176,4408.612793,92.428139,...,0,1,1,0,1,0,0,0,1,0
997,0.537085,58.211857,24.820795,-2.690068,-10.519603,699.137634,3392.163086,3834.084473,4250.133301,-249.118576,...,0,1,1,0,1,1,0,0,1,0
998,0.359891,58.364864,5.214003,-2.313634,-11.050214,702.931641,2810.132080,3352.084961,3977.682617,-183.449387,...,0,1,1,0,1,1,0,0,1,0


In [9]:
def reverse_one_hot_encoding(df, categorical_columns):
    df_reversed = df.copy()
    
    for col in categorical_columns:
        col_prefix = col + '_'
        category_cols = [c for c in df.columns if c.startswith(col_prefix)]
        
        if not category_cols:
            # Eğer bu kategorik sütun için one-hot encoded sütun bulunmazsa devam et
            continue
        
        # One-hot encoded sütunlardan kategorik veriyi geri döndür
        df_reversed[col] = np.nan
        for category_col in category_cols:
            category_value = category_col.replace(col_prefix, '')
            df_reversed.loc[df_reversed[category_col] == 1, col] = category_value
            
        # One-hot encoded sütunları düşür
        df_reversed.drop(columns=category_cols, inplace=True)
        
    return df_reversed

# Örnek kullanım
original_df = reverse_one_hot_encoding(generated_df, binary_columns)


dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd
dd


In [8]:
original_df

,name_email_similarity,prev_address_months_count,current_address_months_count,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,...,month_5,month_6,month_7,payment_type_AA,payment_type_AB,payment_type_AC,payment_type_AD,payment_type_AE,source_INTERNET,source_TELEAPP
0,0.258326,55.100857,73.546875,-0.730430,6.918195,1304.980835,8024.230957,5844.652344,5546.629883,-238.667709,...,0,1,0,1,0,1,0,0,1,0
1,0.320405,56.863998,34.139175,-3.391500,-10.188081,1888.504517,2954.628906,3363.331055,4091.190918,57.797241,...,0,1,1,0,1,0,0,1,1,0
2,0.516030,38.475037,131.727371,-3.190106,-0.940820,2028.784912,4159.578613,4887.316895,4604.526367,37.729118,...,0,1,1,0,1,0,0,0,1,0
3,0.486139,60.317142,5.642218,-3.792231,-11.002547,1291.702271,3287.737793,3748.450928,4903.172852,-248.362808,...,0,1,1,0,1,0,0,1,1,0
4,0.657974,17.441010,171.707474,-4.487044,-1.563043,2558.560791,2742.557129,3840.071533,4242.323730,389.209991,...,0,1,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.618908,60.605434,8.352049,-3.842474,-11.272525,621.627136,2879.943604,3462.335205,4106.711426,-271.491882,...,0,1,1,0,1,1,0,0,1,0
996,0.524920,27.115891,162.356964,-3.967557,-4.156897,2317.308350,3324.681152,4132.930176,4408.612793,92.428139,...,0,1,1,0,1,0,0,0,1,0
997,0.537085,58.211857,24.820795,-2.690068,-10.519603,699.137634,3392.163086,3834.084473,4250.133301,-249.118576,...,0,1,1,0,1,1,0,0,1,0
998,0.359891,58.364864,5.214003,-2.313634,-11.050214,702.931641,2810.132080,3352.084961,3977.682617,-183.449387,...,0,1,1,0,1,1,0,0,1,0


In [14]:
df1 = pd.read_csv("sample10.csv")

In [16]:
for col in df.columns:
    if col in original_df.columns:
        original_df[col] = original_df[col].astype(df1[col].dtype)

KeyError: 'income_0.1'

In [ ]:
final_df.head()

In [ ]:
df1.head()

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Veri setinizi yükleyin
df = one_hot_encoded_df

# Veri ön işleme
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data.values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Özelliklerinizi ayırın
y = df['fraud_bool']  # Etiketler
X = df.drop(columns=['fraud_bool'])  # Özellikler

# Verileri standardize edin
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Eğitim ve test setlerine ayırın
X_train, X_test, _, _ = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# PyTorch veri yükleyicilerini oluşturun
train_dataset = CustomDataset(pd.DataFrame(X_train))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# GAN modeli tanımlamaları
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.fc(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.fc(x)

# GAN'ı oluşturun
latent_dim = 100
input_dim = X_train.shape[1]

generator = Generator(latent_dim, input_dim)
discriminator = Discriminator(input_dim)

criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# GAN eğitimi
num_epochs = 20
for epoch in range(num_epochs):
    for real_data in train_loader:
        batch_size = real_data.size(0)

        # Gerçek ve sahte veriler için etiketler
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        # Discriminator'ı eğitme
        optimizer_d.zero_grad()
        outputs = discriminator(real_data)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        z = torch.randn(batch_size, latent_dim)
        fake_data = generator(z)
        outputs = discriminator(fake_data.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()
        optimizer_d.step()

        # Generator'ı eğitme
        optimizer_g.zero_grad()
        outputs = discriminator(fake_data)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_g.step()

    print(f'Epoch [{epoch+1}/{num_epochs}] | D Loss: {d_loss_real.item() + d_loss_fake.item()} | G Loss: {g_loss.item()}')

# Yapay veri üretme
z = torch.randn(1000, latent_dim)
generated_data = generator(z).detach().numpy()
generated_data = scaler.inverse_transform(generated_data)

# Yapay veriyi DataFrame'e dönüştürme
generated_df = pd.DataFrame(generated_data, columns=df.columns[:-1])

